### This file is a playground for developing some functions and checking them before putting them in the .py files

##### Develop function to get a 3d-patch around some pixel and get its class from the needle mask h5 file

In [3]:
# import all files from necessary directory
from utils.labels_utils import get_all_files_mhd
all_files_mhd = get_all_files_mhd("/data/Riabova/train/train_depth_0_70/")
len(all_files_mhd)

110

In [4]:
from tqdm import tqdm
import h5py
from utils.type_reader import get_image_array, mha_read_header
from utils.mask_utils import get_needle_mask_of_frame_sequence
from utils.labels_utils import get_labels
import numpy as np

filename = all_files_mhd[0]
f = filename[:-4].split("/")[-1]
new_file = h5py.File('../train/needle_masks_h5/'+f+'.hdf5', 'w')
sequence_4d = get_image_array(filename)
info = mha_read_header(filename)
labels = get_labels(f, info)
needle_mask_sequence = get_needle_mask_of_frame_sequence(sequence_4d, labels)
new_file.create_dataset("default", data=np.transpose(needle_mask_sequence,(3, 0, 1, 2)))
new_file.close()

 43%|████▎     | 5576692/12977875 [01:28<01:55, 63953.42it/s]

In [ ]:
# create and return 3 2d-patches around some pixel
# patch size has to be 1)odd to make the pixel in question to be exactly in the center, 2) divisible by number of layers of network
def get_2d_patches(filename, frame_num, pixel_coords, patch_size=21):
    f = filename[:-4].split("/")[-1]
    h5_file = h5py.File('../train/trainh5/'+f+'.hdf5', 'r')
    input_image = h5_file['default'][frame_num]
    h5_file.close()
    patch_xy = input_image[(pixel_coords[0]-(patch_size-1)/2):(pixel_coords[0]+(patch_size-1)/2), (pixel_coords[1]-(patch_size-1)/2):(pixel_coords[1]+(patch_size-1)/2), pixel_coords[2]]
    patch_yz = input_image[pixel_coords[0], (pixel_coords[1]-(patch_size-1)/2):(pixel_coords[1]+(patch_size-1)/2), (pixel_coords[2]-(patch_size-1)/2):(pixel_coords[2]+(patch_size-1)/2)]
    patch_xz = input_image[(pixel_coords[0]-(patch_size-1)/2):(pixel_coords[0]+(patch_size-1)/2), pixel_coords[1], (pixel_coords[2]-(patch_size-1)/2):(pixel_coords[2]+(patch_size-1)/2)]
    return [patch_xy, patch_xz, patch_yz]

In [ ]:
# define the class of the pixel (0 - not needle pixel, 1-needle pixel)
def get_pixel_class(filename, frame_num, pixel_coords):
    f = filename[:-4].split("/")[-1]
    needle_mask_h5_file = h5py.File('../train/needle_masks_h5/'+f+'.hdf5', 'r')
    needle_mask = needle_mask_h5_file['default'][frame_num]
    needle_mask_h5_file.close()
    return needle_mask[pixel_coords]
